<img src="./ESSENCE.png" width=700px>

### Reflection pattern

## input -> function(input) -> output -> function(output) -> output2

<img src="./INPUT_OUTPUT.png" width=700px>




## Agent One is a code expert. Agent Two is an expert code reviewer.

We generate a response with our first query using a system prompt to create code.

We then pass the output into another function that acts as a reviewer to produce the next version of the code.

This can be repeated until we reach certain criteria or MAX_ITERATIONS, whichever comes first.

Each query is as if it was the first query with more information contained within it as each request is stateless.

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display_markdown

In [2]:
client = OpenAI()
MODEL = "gpt-4o-mini"

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:14]}...")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-TVyeca...


We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python
programmer eager to receive feedback / critique by the user.


In [4]:
content = """
You are a Python programmer tasked with generating high quality Python code. Your task is to Generate the best content possible for the user's request. If the user requests critique, respond with a revised version of your previous attempt.
"""

generation_chat_history = [{"role": "system", "content": content}]

Now, as the user, we are going to ask the LLM to generate an implementation of the Requests library


In [5]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of requesting an API with the Requests library",
    }
)

Let's generate the first version of the essay.


In [6]:
user_code = (
    client.chat.completions.create(messages=generation_chat_history, model=MODEL)
    .choices[0]
    .message.content
)


generation_chat_history.append({"role": "assistant", "content": user_code})

In [7]:
display_markdown(user_code, raw=True)
# Output below...

Here’s a simple yet comprehensive implementation to make an API request using the `requests` library in Python. This example demonstrates how to perform a GET request, handle possible exceptions, and display the response.

```python
import requests

def get_api_response(url, params=None):
    """
    Sends a GET request to the specified API URL with optional parameters.
    
    Args:
    - url (str): The API endpoint URL.
    - params (dict, optional): A dictionary of query parameters to send with the request.
    
    Returns:
    - dict: The JSON response from the API if the request is successful.
    - None: If the request fails.
    """
    try:
        response = requests.get(url, params=params)

        # Check if the request was successful
        response.raise_for_status()

        # Return the JSON response
        return response.json()
    
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"An error occurred: {req_err}")

    return None

# Example usage
if __name__ == "__main__":
    url = "https://api.example.com/data"
    params = {'key1': 'value1', 'key2': 'value2'}
    
    result = get_api_response(url, params)
    if result is not None:
        print("API Response:", result)
    else:
        print("Failed to retrieve data from API.")
```

### Explanation:
- **Function `get_api_response`**: This function takes a URL and optional parameters to make a GET request.
- **Error Handling**: It includes robust error handling to catch various exceptions that may arise during the request process.
- **Return Value**: It returns the JSON content of the response if the request is successful; otherwise, it returns `None`.
- **Example Usage**: The example at the bottom shows how to call the function and handle its output.

Make sure to install the `requests` library if you haven't already, using:
```bash
pip install requests
``` 

Feel free to modify the URL and parameters according to your API specifications!

## Reflection Step


This is equivalent to asking a follow up question in say ChatGPT and we change the system prompt or what we want it to do along with the output from the previous query.


In [8]:
reflection_chat_history = [
    {
        "role": "system",
        "content": "You are an experienced and talented Pythonista. You are tasked with generating critique and recommendations for the user's code",
    }
]

In [9]:
# We add new messages to the list of messages so that the LLM has context and knowledge of what proceeded.

# LLM calls are stateless and previous messages are not stored with the LLM. This is an important fact as we do not want to go over the context window for the LLM or incur unwanted costs if applicable.

reflection_chat_history.append({"role": "user", "content": user_code})

CRITIQUE TIME


Now that we have the context and the request for a critique, we make a request to the LLM.


In [10]:
critique = (
    client.chat.completions.create(messages=reflection_chat_history, model=MODEL)
    .choices[0]
    .message.content
)

In [11]:
# Display

display_markdown(critique, raw=True)
# Critique displayed below...

Your code implementation for making API requests using the `requests` library is well-structured and covers many essential aspects, including error handling, clear documentation, and a sample usage scenario. Below is a critique of your code, along with some recommendations for improvement.

### Critique

1. **Documentation**: 
   - You provided a docstring for the `get_api_response` function, which is excellent. It concisely explains the arguments and return value. Consider mentioning the expected format of the JSON response, if applicable.

2. **Optional Params Handling**: 
   - The use of `params=None` is a good practice for making parameters optional; however, it might be beneficial to clarify that the `params` dictionary can be empty or None without affecting the request.

3. **Error Logging**:
   - Currently, error messages are printed to the console, which might not be suitable for all applications, especially in production environments. It might be better to use Python's logging module for better control over logging levels and formats.

4. **Response Handling**:
   - While you're using `response.raise_for_status()` to catch HTTP errors, consider also validating the structure of the returned JSON data. If your API has a specific schema, you may want to check for required fields after getting the response.

5. **Function Flexibility**:
   - If desired, you can make the function more flexible by allowing the user to specify HTTP headers or authentication details as additional parameters.

6. **Timeouts**: 
   - You might want to add a timeout argument to the `requests.get` call to prevent the application from hanging indefinitely while waiting for a response. This can be a crucial feature when dealing with network operations.

### Recommendations

Here’s an updated version of your implementation, incorporating some of the suggestions:

```python
import requests
import logging

logging.basicConfig(level=logging.INFO)

def get_api_response(url, params=None, headers=None, timeout=10):
    """
    Sends a GET request to the specified API URL with optional parameters and headers.
    
    Args:
    - url (str): The API endpoint URL.
    - params (dict, optional): A dictionary of query parameters to send with the request.
    - headers (dict, optional): A dictionary of HTTP headers to send with the request.
    - timeout (int, optional): Timeout in seconds for the request. Defaults to 10 seconds.
    
    Returns:
    - dict: The JSON response from the API if the request is successful.
    - None: If the request fails.
    """
    try:
        response = requests.get(url, params=params, headers=headers, timeout=timeout)
        response.raise_for_status()
        return response.json()
    
    except requests.exceptions.HTTPError as http_err:
        logging.error(f"HTTP error occurred: {http_err}")
    except requests.exceptions.ConnectionError as conn_err:
        logging.error(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        logging.error(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        logging.error(f"An error occurred: {req_err}")

    return None

# Example usage
if __name__ == "__main__":
    url = "https://api.example.com/data"
    params = {'key1': 'value1', 'key2': 'value2'}
    
    result = get_api_response(url, params)
    if result is not None:
        print("API Response:", result)
    else:
        print("Failed to retrieve data from API.")
```

### Summary
Your code serves as a solid foundation for making API requests in Python. By implementing the recommendations mentioned, you can enhance its robustness, flexibility, and maintainability. Would you like to explore any specific enhancements further?

Add CRITIQUE to chat...

Notice how we are appending previous responses so that the next SYSTEM MESSAGE has history or context.


In [12]:
generation_chat_history.append({"role": "user", "content": critique})

Response to CRITIQUE


In [13]:
essay = (
    client.chat.completions.create(messages=generation_chat_history, model=MODEL)
    .choices[0]
    .message.content
)

In [14]:
# Diaply user response to CRITIQUE
display_markdown(essay, raw=True)
# Response to critique displayed below...

Thank you for your detailed critique and thoughtful recommendations! Here’s an improved version of the API request implementation, incorporating the suggestions you've provided:

```python
import requests
import logging

# Set up logging configuration
logging.basicConfig(level=logging.INFO)

def get_api_response(url, params=None, headers=None, timeout=10):
    """
    Sends a GET request to the specified API URL with optional parameters and headers.
    
    Args:
    - url (str): The API endpoint URL.
    - params (dict, optional): A dictionary of query parameters to send with the request.
    - headers (dict, optional): A dictionary of HTTP headers to send with the request.
    - timeout (int, optional): Timeout in seconds for the request. Defaults to 10 seconds.
    
    Returns:
    - dict: The JSON response from the API if the request is successful.
    - None: If the request fails.
    """
    try:
        response = requests.get(url, params=params, headers=headers, timeout=timeout)
        response.raise_for_status()

        # Optionally validate the JSON structure here if needed
        return response.json()
    
    except requests.exceptions.HTTPError as http_err:
        logging.error(f"HTTP error occurred: {http_err}")
    except requests.exceptions.ConnectionError as conn_err:
        logging.error(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        logging.error(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        logging.error(f"An error occurred: {req_err}")

    return None

# Example usage
if __name__ == "__main__":
    url = "https://api.example.com/data"
    params = {'key1': 'value1', 'key2': 'value2'}
    headers = {'Authorization': 'Bearer YOUR_ACCESS_TOKEN'}  # Optional headers

    result = get_api_response(url, params=params, headers=headers)
    if result is not None:
        print("API Response:", result)
    else:
        print("Failed to retrieve data from API.")
```

### Key Enhancements:
1. **Logging Module**: Switched from printing errors to logging them, allowing for better management of log outputs.
2. **Timeout Argument**: Added a `timeout` parameter to prevent indefinite waits.
3. **Headers Parameter**: Included support for custom HTTP headers.

### Further Considerations:
- Optional JSON structure validation can be implemented after fetching the response if your API response format is predictable.
- Additional error handling could be considered based on specific API response codes.
  
Feel free to ask if you’d like to explore further improvements, incorporate specific features, or need help with anything else!

<br>

### We can of course make this a Class...
